In [3]:
import pdfplumber
import re
import os
import pandas as pd

def invoice_df(path):
    doc_lst = os.listdir(path)
    doc_lst.sort()
    full_df = pd.DataFrame()
    for doc in doc_lst:
        file = os.path.join(path, doc)
        pdf = pdfplumber.open(file)
        df = pd.DataFrame()
        for item in pdf.pages:
            table=item.extract_table()
            dff = pd.DataFrame(table[1::],columns=table[0])
            dff = dff[dff.isnull().sum(axis=1) < 3]
            df = pd.concat([df, dff])
        df.reset_index(inplace = True, drop=True)
        page = pdf.pages[0]
        text = page.extract_text()
#         print(text)
        document_number = r'№\s+\d+'
        date_pattern = r'[\d]{1,2}.[\d]{1,2}.[\d]{4}'

        document_number = re.findall(document_number, text)
        date_pattern = re.findall(date_pattern, text)
        df['Փաստ №'] = df.apply(lambda x: document_number[0], axis=1)
        df['Ամսաթիվ'] = df.apply(lambda x: date_pattern[3], axis=1)
        
        sub1 = "Մատակարարը"
        sub2 = "Առաքման գրանցման գիրք 3"
        s=str(re.escape(sub1))
        e=str(re.escape(sub2))
        entity=re.findall(s+"(.*)"+e,text)[0]
        
        df['Կազմակերպություն'] = df.apply(lambda x: entity, axis=1)
        
        full_df = pd.concat([full_df, df])
    full_df.reset_index(inplace = True, drop=True)
    return full_df

In [4]:
path = r'C:\Users\svardanyan\Desktop\termoar\invoices'

df = invoice_df(path)
df

,#,Կոդը,Անվանումը,Չափման\nմիավոր,Քանակը,Գինը,Գումարը,Փաստ №,Ամսաթիվ,Կազմակերպություն
0,1,C-MT,Պղնձե խողոովակի մոնտաժ,մ,140,1770.83,None,№ 101,29.04.2022,"ԹԵՐՄՈՌՈՍ-ԱՐ ՍՊԸ, ՀՎՀՀ 02566649"
1,2,QAND,Քանդման աշխատանքներ,հատ,30,1416.67,42500.00,№ 101,29.04.2022,"ԹԵՐՄՈՌՈՍ-ԱՐ ՍՊԸ, ՀՎՀՀ 02566649"
2,3,SIF-01,Սիֆոնի մոնտաժ,հատ,1,7083.33,7083.33,№ 101,29.04.2022,"ԹԵՐՄՈՌՈՍ-ԱՐ ՍՊԸ, ՀՎՀՀ 02566649"
3,4,JHR,Ջրահեռացում,մ,35,1416.67,49583.33,№ 101,29.04.2022,"ԹԵՐՄՈՌՈՍ-ԱՐ ՍՊԸ, ՀՎՀՀ 02566649"
4,5,PR04,"Այլ ծախսեր, ապրանքներ",հատ,1,540458.33,540458.33,№ 101,29.04.2022,"ԹԵՐՄՈՌՈՍ-ԱՐ ՍՊԸ, ՀՎՀՀ 02566649"
...,...,...,...,...,...,...,...,...,...,...
353,55,APSTX20B,Պրես–պարկուճ 20,հատ,5,389.58,1947.92,№ 961,29.03.2022,"ԹԵՐՄՈՌՈՍ-ԱՐ ՍՊԸ, ՀՎՀՀ 02566649"
354,56,2400081620200,Սպիտակ մետաղապլաստե խողովակ 16x2,մ,106,531.25,56312.50,№ 961,29.03.2022,"ԹԵՐՄՈՌՈՍ-ԱՐ ՍՊԸ, ՀՎՀՀ 02566649"
355,57,2550001620100,Սպիտակ մետաղապլաստե խողովակ մեկուսիչով 16x2,մ,66,779.17,51425.00,№ 961,29.03.2022,"ԹԵՐՄՈՌՈՍ-ԱՐ ՍՊԸ, ՀՎՀՀ 02566649"
356,58,2550002020050,Սպիտակ մետաղապլաստե խողովակ մեկուսիչով 20x2,մ,50,1204.17,60208.33,№ 961,29.03.2022,"ԹԵՐՄՈՌՈՍ-ԱՐ ՍՊԸ, ՀՎՀՀ 02566649"


In [ ]:
df.Ամսաթիվ[0]